# Data Preprocessing

In [1]:
# Import libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

import tabula

import os

from PyPDF2 import PdfReader
import pandas as df

In [2]:
# load in openai API key
os.environ['OPENAI_API_KEY'] =""

## Document used and test cases

For the purpose of experimentation, let's use the Tesla Annual Report for year 2022. This document contains both text and tabular information (at the end of document), hence it will be a good test document for our prompt engineering. The document is stored in the "./data" folder.

We should also include some basic test cases beforehand. This will make it easy to check if the prompts are giving us the correct output. Some test cases are defined here for reference.

* Numbers. To test tabular data. 

1. What are the total assets and liabilities for the year 2022? Answer: 82338 million and 36440 million.
1. What is the Income before income taxes for the year  2020, 2021 and 2022? Answer: 1,154, 6,343 and 13,719 respectively.
1. What is the year on year increase in the gross profit from 2020 to 2022 in percentages? Answer: 105.2% and 53.3% respectively.
Note: This tests the higher level logical thinking of the model.
1. What is the gross profit in 2019?
Note: This tests if the model will give a truthful answer or hallucinates the numbers.

* General Questions (open-ended)
1. What does Tesla do? And what types of businesses is Tesla involved in?
1. How is the financial health of Tesla?
1. What kind of technologies does Tesla invest in?

In [11]:
# template code to be used for testing of prompts
query_1 = "What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?"
query_2 = "What is the Income before income taxes for the year  2020, 2021 and 2022?"
query_3 = "What is the year on year increase in the gross profit from 2020 to 2022 in percentages?"
query_4 = "What is the gross profit in 2019?"
queries_num = [query_1,query_2,query_3,query_4]

query_5 = "What does Tesla do? And what types of businesses is Tesla involved in?"
query_6 = "How is the financial health of Tesla?"
query_7 = "What kind of technologies does Tesla invest in?"
queries_gen = [query_5, query_6, query_7]

## Verify tabular data integrity

As the tables are all converted to a string of text before passing to the embedding model, we need to check if the model can still understand the structured information.

In [22]:
# read in the information from the Tesla Annual Report, convert to embeddings and store in vectorestore
# using Langchain PyPDFLoader
# Note: PyPDFLoader is for structured data extraction
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
print (f'You have {len(data)} pages in your pdf')

You have 184 pages in your pdf


In [23]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(data, embeddings)

In [10]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())
for query in queries_num:
    print(query)
    print(qa.run(query))
    print("---------------------------------")

What are the total assets and liabilities for the year 2022?
The total assets for the year 2022 are $82,338 million. The total liabilities for the year 2022 are $36,440 million.
---------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the years 2020, 2021, and 2022 are as follows:

- 2020: $1,154 million
- 2021: $6,343 million
- 2022: $13,719 million
---------------------------------
What is the year on year increase in the gross profit from 2020 to 2022 in percentages?
The information provided does not directly state the gross profit for the years 2020 and 2022. Therefore, I cannot determine the year-on-year increase in gross profit from 2020 to 2022 in percentages.
---------------------------------
What is the gross profit in 2019?
The given context does not provide information about the gross profit in 2019.
---------------------------------


**Observations**
* The model is able to read the tabular data that has been read in and embedded. This is what we want to verify.
* This is what we expected as PyPDFLoader is a structured data extraction. Tables are considered as structured data.
* The base embedding and chat model is already quite good. It can give accurate information and does not give answers it does not know.
* On higher level tasks, it seems to lose the contextual information.
* Some of the information like the gross profit are not included.

Let's try a different package to read in the data. The package is PyPDF2 --> PdfReader. Note that PyPDF2 is inherently for structured data extraction.

In [131]:
def get_text_from_pdf(fs_pdf_docs: list):
    text_output = ""
    for pdf_file in fs_pdf_docs:
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text_output = text_output + page.extract_text()
    return text_output

def get_chunk_from_text(whole_text: str):
    text_split = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " ", ""],
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len
    )
    chunks = text_split.split_text(whole_text)
    return chunks

def get_vectorstore_from_chucks(chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts = chunks, embedding = embeddings)
    return vectorstore

In [132]:
text = get_text_from_pdf([FILE_PATH])
chunks = get_chunk_from_text(text)
vector_store = get_vectorstore_from_chucks(chunks)

In [51]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())
for query in queries_num:
    print(query)
    print(qa.run(query))
    print("---------------------------------")

What are the total assets and liabilities for the year 2022?
The total assets for the year 2022 are $82,338 million. The total liabilities for the year 2022 are $36,440 million.
---------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the years 2020, 2021, and 2022 are as follows:

- 2020: $1,154 million
- 2021: $6,343 million
- 2022: $13,719 million
---------------------------------
What is the year on year increase in the gross profit from 2020 to 2022 in percentages?
The year-on-year increase in gross profit from 2020 to 2022 cannot be determined based on the provided information. The gross profit figures for 2020 are not given in the context.
---------------------------------
What is the gross profit in 2019?
The information provided does not include the gross profit for the year 2019.
---------------------------------


**Observations:**
* The output using PyPDF2 is the same as using PyPDFLoader. This is also expected as PdfReader is a structured data extractor.
* Hence, we can use PyPDF2 for the app as it allows for file type object loading with is not possible with PyPDFLoader.

## Secondary reference for tabular data
To provide a secondary module to extract the tabular data and acts as a source of reference to ground the context for the prompts.
methods:
1. Use prompt to extract to extract tabular data
1. Use camelot package to extract tabular data

In [123]:
# # creating a prompt to extract the tables as a csv format for further grounding of the information
# query = "Using the context provided, please extract all the tabular data in the report and output the tabular data in a csv file format"
# qa = load_qa_chain(llm=ChatOpenAI(temperature = 0.1), chain_type="map_reduce")
# qa.run(input_documents=data, question=query)

# Does not work, take too long for map_reduce. "stuff" exceeds the max token length


In [135]:
tables = camelot.read_pdf(FILE_PATH, flavor='stream', pages ="all")

/home/jethro/anaconda3/envs/langchain/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/jethro/anaconda3/envs/langchain/lib/python3.11/site-packages/camelot/parsers/stream.py:453: UserWarning: No tables found on page-2
  warnings.warn(f"No tables found on {base_filename}")
/home/jethro/anaconda3/envs/langchain/lib/python3.11/site-packages/camelot/parsers/stream.py:453: UserWarning: No tables found on page-183
  warnings.warn(f"No tables found on {base_filename}")


In [154]:
print(len(tables))
tables[190].df

222


,0,1,2,3,4,5,6,7,8
0,Note 14 – Income Taxes,,,,,,,,
1,,"A provision for income taxes of $1.13 billion,...",,,,,,,
2,,"December 31, 2022, 2021 and 2020, respectively...",,,,,,,
3,,provision for income taxes for the years ended...,,,,,,,
4,,,,"Year Ended December 31,",,,,,
5,,,2022,,2021,,,2020,
6,Domestic,$,"5,524",$,,(130),$,,(198)
7,Noncontrolling interest and redeemable,,,,,,,,
8,noncontrolling interest,,31,,,125,,,141
9,Foreign,,"8,164",,,"6,348",,,"1,211"


**Observations:**
* As can be seen, there are 222 tables generated, which is a lot.
* Upon inspection, it is found that there are many tables which are actually general tables and not relevant to financial data.
* Hence, we need a search function to find only relevant pages to pass to the camelot module for table extraction


In [192]:
# read in the information from the Tesla Annual Report, convert to embeddings and store in vectorestore
# using Langchain PyPDFLoader
# Note: PyPDFLoader is for structured data extraction
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
print (f'You have {len(data)} pages in your pdf')
type(data[0].page_content)

You have 184 pages in your pdf


str

In [190]:
def find_financial_report(documents):
    page_num_string = ""
    search_words = ["balance sheet", "income statement", "statement of cash flow", "statement of retained earnings"]
    for index, page in enumerate(documents):
        string_content = page.page_content.lower()
        if any(map(string_content.__contains__, search_words)):
            page_num_string += str(index+1)+","
    if page_num_string[-1]==",":
        page_num_string = page_num_string.rstrip(page_num_string[-1])
    return page_num_string


In [193]:
page_number = find_financial_report(data) # we now have a string of pages that we can pass to camelot to extract the relevant data

In [195]:
tables = camelot.read_pdf(FILE_PATH, flavor='stream', pages =page_number)
len(tables)

/home/jethro/anaconda3/envs/langchain/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


28

In [201]:
tables[24].df

,0,1,2,3,4
0,,The balances for the operating and finance lea...,,,
1,consolidated balance sheets:,,,,
2,,,"December 31, 2022",,"December 31, 2021"
3,Operating leases:,,,,
4,Operating lease right-of-use assets,$,"2,563",$,"2,016"
5,Accrued liabilities and other,$,485,$,368
6,Other long-term liabilities,,"2,164",,"1,671"
7,Total operating lease liabilities,$,"2,649",$,"2,039"
8,Finance leases:,,,,
9,"Solar energy systems, net",$,25,$,27


In [208]:
tables[19].df

,0,1,2,3
0,"Our solar energy systems, net, consisted of th...",,,
1,,,"December 31,","December 31,"
2,,,2022,2021
3,Solar energy systems in service,$,"6,785","$\n6,809"
4,Initial direct costs related to customer solar...,,,
5,system lease acquisition costs,,104,104
6,,,"6,889","6,913"
7,Less: accumulated depreciation and amortizatio...,,"(1,418)","(1,187)"
8,,,"5,471","5,726"
9,Solar energy systems under construction,,2,18


In [196]:
# store the text of the dataframe into the vectorstore as embeddings
df_list = []
for table in tables:
    df = table.df
    text = df.to_string()
    df_list.append(text)
vectorstore = FAISS.from_texts(texts = df_list, embedding = embeddings)

**Observations:**
* Many of the tables are missing the headers and the "millions". There are also still many incomplete tables.
* This could cause the model performance to degrade due to inaccurate data being added to the vector store.
* Further effort is needed to properly extract the tables from the pdf documents. Can try other packages.
* Perhaps also can consider CV models for extracting tabular information.

## Module to check if document is indeed a financial report or not

To pass a prompt to classify the financial report as true or false. If the documents passed into the model are indeed not financial reports, to then give a warning to user so that he/she knows the output might not be correct.

In [12]:
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
print (f'You have {len(data)} pages in your pdf')

You have 184 pages in your pdf


In [25]:
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(data, embeddings)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())

In [26]:
# inspect the default template in RetreievalQA
qa.combine_documents_chain

StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})]), llm=ChatOpe

In [54]:
system_template = """Use the following pieces of context to answer the users question.\
No matter what the question is, you should always answer it as truthfully and accurately as possible\\
If you don't know the answer, just say that "I don't know", don't try to make up an answer.\
Your task is to classify if the report is a financial report or not a financial report.\
Again, your task is to classify if the report is a financial report or not a financial report.\
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}\Classification:"),
]
prompt = ChatPromptTemplate.from_messages(messages)
chain_type_kwargs = {"prompt": prompt}

In [41]:

qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0.0), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs=chain_type_kwargs)
query = "Please classify if the report is a financial report or not a financial report"
qa.run(query)


'.\n\nNo, this is not a financial report.'

Let's gather 5 reports, of which 3 are financial report and 2 are not financial reports. To observe if the LLM can actually discern it accurately.

In [42]:
DOC_CC = "../data/Cocacola_earningsrelease2022.pdf"
DOC_DESSERT = "../data/Dessert Lovers Foodbook 2016.pdf"
DOC_DISNEY = "../data/Disney_2022-Annual-Report.pdf"
DOC_FLATTIRE = "../data/How-to-Change-a-Flat-Tire-Student-Handout.pdf"
DOC_TESLA = "../data/Tesla_Annual_Report_2022.pdf"

In [43]:
def classify_report(filepath, chain_type_kwargs):
    loader = PyPDFLoader(filepath)
    data = loader.load()

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(data, embeddings)
    qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs=chain_type_kwargs)

    query = "Please classify if the report is a financial report or not a financial report"
    print(filepath)
    return qa.run(query)

In [55]:
doc_list = [DOC_CC, DOC_DESSERT, DOC_DISNEY, DOC_FLATTIRE, DOC_TESLA]
for report in doc_list:
    print(classify_report(report, chain_type_kwargs))
    print("---------------------------------------")

../data/Cocacola_earningsrelease2022.pdf
Financial report
---------------------------------------
../data/Dessert Lovers Foodbook 2016.pdf
Not a financial report
---------------------------------------
../data/Disney_2022-Annual-Report.pdf
Financial report
---------------------------------------
../data/How-to-Change-a-Flat-Tire-Student-Handout.pdf
Not a financial report
---------------------------------------
../data/Tesla_Annual_Report_2022.pdf
Financial report
---------------------------------------


**Observations:**
* After some tweaking of the prompt, the LLM is able to give consistently correctly answers on whether the report is or is not a financial report.

## Input extra relevant information in the vector store embeddings for use as context
It will be good to improve the context that the model has access to, especially for financial report terms and definitions.
Here, we add the internal document with the information of finance terms into the vector store embeddings to be used by the LLM.

In [39]:
# tesla document embeddings
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(data, embeddings)

In [40]:
# financial terms embedding
FILE_PATH = "../data/Glossary of financial terms.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
embeddings = OpenAIEmbeddings()
vbase = FAISS.from_documents(data, embeddings)

In [41]:
vbase.save_local("../faiss store/financial_terms_embedding")

In [42]:
vbase_load = FAISS.load_local("../faiss store/financial_terms_embedding", embeddings)

In [43]:
vbase_load.merge_from(vector_store)

In [47]:
final_store = vbase_load

In [49]:
final_store.docstore._dict

{'f61bfd22-ae57-4150-b619-d9ef8b504bde': Document(page_content='Services, Projects & Operations » Financial Management » Essential Information » Glossary of Financial Terms\nGlossary of Financial Terms\nHave you ever found yourself a little bamboozled by the terminology used in financial activities? If so, use this glossary to\nkeep you on track.\nBasic Financial Terms\nSome of the more straightforward financial terms that you might come across are listed below.\nAccounting Manual:\n A document that details the accounting policies and procedures of a business. It normally contains\na list of account codes or chart of accounts. The way a company treats depreciation is an example of an accounting\npolicy which would be set out in the accounting manual.\nAccounting Period:\n The period for which a company prepares its accounts. Management accounts may be produced\ninternally on a monthly or quarterly basis, whilst financial accounts will be made for a period of one year. An accounting\npe